## Hovmoller

 A Hovmöller shows how a variable evolves **through time** and **down the water column** at the nearest model node/element to each `(name, lat, lon)` in `STATIONS`.

Figures are written under:

```
FIG_DIR/<basename(BASE_DIR)>/
```

…with filenames like:

```
<prefix>__Hovmoller-Station-<NAME>__<VAR>__sigma|z__<TimeLabel>.png
```

---

###  What you control (high level)

* **Stations**

  * The code resolves the **nearest grid node/element** by great-circle distance.

* **Axis (vertical)**

  * `axis="sigma"` — plot native σ layers (unitless). No interpolation.
  * `axis="z"` — plot **absolute depth** (m, negative downward) by interpolating σ-profiles to regular `z_levels`.

    * If `z_levels` is **omitted**, a sensible set is auto-built from the column’s min depth to 0 m.

* **Variables**

  * Native fields (e.g., `temp`) or **groups** from `GROUPS` (e.g., `chl`, `phyto`, `DOC`).

* **Time window**

  * `months=[…]`, `years=[…]`, or `start_date="YYYY-MM-DD"`, `end_date="YYYY-MM-DD"`.
  * A compact **time label** is appended to filenames.

* **Styling & scales**

  * Colormap via `cmap` or per-var in `PLOT_STYLES` (e.g., `{"chl": {"cmap": "Greens"}, "chl": {"norm": LogNorm(...)}}`).
  * If no `norm` or `vmin/vmax`, limits are chosen **robustly** from the data.
  * `dpi`, `figsize` control output look; `verbose` toggles printouts.

---

###  Usage

1. Time-filter `ds` using your window.
2. For each station, pick nearest **node/element** and extract that **single column**.
3. Choose vertical axis:

   * **σ**: direct `time × siglay` pcolormesh.
   * **z**: compute vertical coordinates (`ensure_z_from_sigma`), interpolate each profile to `z_levels`, then pcolormesh.
4. Compute colour limits (norm → explicit → robust) and **save** the figure.




In [ ]:
# Setup

BASE_DIR = "/data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var"
FILE_PATTERN = "erie_00??.nc"
FIG_DIR = (
    "/data/proteus1/scratch/moja/projects/Lake_Erie/fvcomersem-viz/examples/plots/"
)


STATIONS = [
    ("WE12", 41.90, -83.10),
    ("WE13", 41.80, -83.20),
]


GROUPS = {
    "DOC": "R1_c + R2_c + R3_c + T1_30d_c + T2_30d_c",  # dissolved organic carbon (sum of pools)
    "phyto": ["P1_c", "P2_c", "P4_c", "P5_c"],  # total phytoplankton carbon (sum)
    "zoo": ["Z4_c", "Z5_c", "Z6_c"],  # total zooplankton carbon (sum)
    "chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl",  # total chlorophyll (sum)
}

PLOT_STYLES = {
    "temp": {"line_color": "lightblue", "cmap": "coolwarm"},
    "DOC": {"line_color": "blue", "cmap": "viridis"},
    "chl": {"line_color": "lightgreen", "cmap": "Greens", "vmin": 0.0, "vmax": 5.0},
    "phyto": {"line_color": "darkgreen", "cmap": "YlGn"},
    "zoo": {"line_color": "purple", "cmap": "PuBu"},
}

from fvcomersemviz.io import load_from_base
from fvcomersemviz.utils import out_dir, file_prefix
from fvcomersemviz.plot import (
    info,
    bullet,
    kv,
    list_files,
    summarize_files,
    print_dataset_summary,
)

import numpy as np

bullet("\nStations (name, lat, lon):")
for s in STATIONS:
    bullet(f"• {s}")


#  Discover files
info(" Discovering files")
files = list_files(BASE_DIR, FILE_PATTERN)
summarize_files(files)
if not files:
    print("\nNo files found. Exiting.")
    sys.exit(2)

#  Load dataset
info(" Loading dataset (this may be lazy if Dask is available)")
ds = load_from_base(BASE_DIR, FILE_PATTERN)
bullet("Dataset loaded. Summary:")
print_dataset_summary(ds)

# Where figures will go / filename prefix
out_folder = out_dir(BASE_DIR, FIG_DIR)
prefix = file_prefix(BASE_DIR)
kv("Figure folder", out_folder)

In [ ]:
# --- Hovmöller examples: station time × depth (save + inline preview) ---


from fvcomersemviz.plots.hovmoller import station_hovmoller

# Station Hovmoller
# Full argument reference for station_hovmoller(...)
# Each parameter below is annotated with what it does and accepted values.
# Produces time × depth (Hovmöller) plots at STATIONS; saves PNGs; returns None.

# def station_hovmoller(
#     ds: xr.Dataset,                               # Xarray Dataset with FVCOM–ERSEM output (already opened/combined)
#     variables: List[str],                         # One or more names: native vars (e.g., "temp") or composites (e.g., "chl") if provided in `groups`
#     stations: List[Tuple[str, float, float]],     # Station metadata as (name, lat, lon) in WGS84 decimal degrees
#                                                   #   - lon west of Greenwich is negative (e.g., -83.10)
#                                                   #   - nearest model node/element is selected by great-circle distance (WGS84)
#     *,                                            # Everything after this must be passed as keyword-only (safer, clearer)
#     axis: str = "z",                              # Vertical axis for the plot:
#                                                   #   "sigma" -> y-axis is sigma layers (unitless), no interpolation
#                                                   #   "z"     -> y-axis is absolute depth in meters (negative downward), σ-profiles interpolated to `z_levels`
#     z_levels: Optional[np.ndarray] = None,        # Regular depth levels (ascending, e.g., np.linspace(-30, 0, 61)) used when axis="z".
#                                                   # If None, levels are auto-built from the station column’s min depth to 0 m.
#     months: Optional[List[int]] = None,           # Calendar months to include (1–12) across all years; e.g., [7] or [4,5,6,7,8,9,10]; None = no month filter
#     years: Optional[List[int]] = None,            # Calendar years to include; e.g., [2018] or [2018, 2019]; None = no year filter
#     start_date: Optional[str] = None,             # Inclusive start date "YYYY-MM-DD"; used with end_date; None = open start
#     end_date: Optional[str] = None,               # Inclusive end date   "YYYY-MM-DD"; used with start_date; None = open end
#     base_dir: str,                                # Path to the model run folder; used for output subfolder and filename prefix
#     figures_root: str,                            # Root directory where figures are saved (module subfolder, e.g., "hovmoller/", is created under this)
#     groups: Optional[Dict[str, Any]] = None,      # Composite definitions enabling semantic names in `variables`:
#                                                   #   {"chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"}    # string expression evaluated in ds namespace
#                                                   #   {"phyto": ["P1_c", "P2_c", "P4_c", "P5_c"]}     # list/tuple summed elementwise
#     cmap: str = "viridis",                        # Default colormap (overridden per-variable by `styles`, if provided)
#     vmin: Optional[float] = None,                 # Explicit lower color limit (ignored if a normalization `norm` is provided via `styles`)
#     vmax: Optional[float] = None,                 # Explicit upper color limit (ignored if a normalization `norm` is provided via `styles`)
#     dpi: int = 150,                               # Output resolution (dots per inch) for saved PNG
#     figsize: tuple = (9, 5),                      # Figure size in inches (width, height)
#     verbose: bool = True,                         # If True, print progress (resolved station index, axis type, time window, file path, etc.)
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Per-variable style overrides:
#                                                   #   {"chl": {"cmap": "Greens", "vmin": 0, "vmax": 5}}
#                                                   #   {"DOC": {"cmap": "viridis"}}
#                                                   #   {"zoo": {"norm": LogNorm(1e-4, 1e0)}}  # norm takes precedence over vmin/vmax
# ) -> None:
#     pass  # Function time-filters ds, resolves nearest node/element per station, builds (time × sigma) or
#           # interpolated (time × z) arrays, chooses color limits (norm -> explicit -> robust), plots pcolormesh,
#           # and SAVES PNG(s); returns None.

# Output path pattern (per station × variable × axis):
#   <figures_root>/<basename(base_dir)>/hovmoller/
#     <prefix>__Hovmoller-Station-<Name>__<VarOrGroup>__sigma|z__<TimeLabel>.png
#
# where:
#   <prefix>    = file_prefix(base_dir)
#   <Name>      = station name from `stations`
#   <TimeLabel> = derived from months/years/start_date/end_date (AllTime, Jul, 2018, 2018-04–2018-10, ...)
#
# Notes:
# - For axis="sigma": plots native σ layers; fastest, no vertical interpolation.
# - For axis="z": vertical coordinates are built (ensure_z_from_sigma); σ-profiles are interpolated to `z_levels`.
# - Station location: nearest grid node (or element) is chosen via great-circle distance in WGS84.
# - If no explicit vmin/vmax/norm, limits are chosen robustly from the plotted data.
# - Returns None; to view in a notebook, display saved PNGs afterward (e.g., using a small gallery cell).


# Example 1: WE12 — chlorophyll on sigma layers (full run, robust colour limits)
station_hovmoller(
    ds=ds,
    variables=["chl"],
    stations=[STATIONS[0]],  # e.g., ("WE12", 41.90, -83.10)
    axis="sigma",
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,  # per-var cmap/norm/vmin/vmax if set
)

# Example 2: WE12 — DOC on absolute depth z (Apr–Oct 2018), explicit z grid
station_hovmoller(
    ds=ds,
    variables=["DOC"],
    stations=[STATIONS[0]],
    axis="z",
    z_levels=np.linspace(-20.0, 0.0, 60),  # omit to auto-build from column depth
    months=[4, 5, 6, 7, 8, 9, 10],  # Apr–Oct
    years=[2018],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
)

# Example 3: WE13 — zooplankton on sigma layers (Apr–Oct 2018)
station_hovmoller(
    ds=ds,
    variables=["zoo"],
    stations=[STATIONS[1]],
    axis="sigma",
    months=[4, 5, 6, 7, 8, 9, 10],  # Apr–Oct
    years=[2018],
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
)

print(" Hovmöller examples completed. Figures saved under:", FIG_DIR)